In [1]:
import os
import logging
import sys
import requests
import time
from contextlib import contextmanager
import textwrap

import faiss
from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader, ServiceContext, LLMPredictor, StorageContext, LangchainEmbedding
from llama_index.vector_stores.faiss import FaissVectorStore
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.huggingface import HuggingFaceEmbeddings


logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)

In [2]:
@contextmanager
def timer(title):
    t0 = time.time()
    print(f"[{title}] - start.")
    yield
    print(f"[{title}] - done in {time.time() - t0 :.1f}s.")

# indexの作成

In [3]:
llm_name = "chatgpt" # "chatgpt", "gpt3"
vector_search_method = "simple"  # "simple", "faiss"
embed_model_name = None  # None, "oshizo/sbert-jsnli-luke-japanese-base-lite"
"""
MEMO: cpu環境なのでembedはやめておく
"""

if llm_name == "gpt3":
    llm = OpenAI(
        model_name="text-davinci-003",
        max_tokens=1024,
        temperature=0,
        frequency_penalty=0.02,
    )
elif llm_name == "chatgpt":
    llm = ChatOpenAI(
        model_name="gpt-3.5-turbo",
        max_tokens=1024,
        temperature=0,
    )

if embed_model_name is None:
    embed_model = None
else:
    embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name=embed_model_name))

documents = SimpleDirectoryReader("data").load_data()
llm_predictor = LLMPredictor(llm=llm)
service_context = ServiceContext.from_defaults(
    llm_predictor=llm_predictor,
    embed_model=embed_model,
)

with timer("make index"):
    if vector_search_method == "simple":
        index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)
    elif vector_search_method == "faiss":
        d = 1536  # dimensions of text-ada-embedding-002
        faiss_index = faiss.IndexFlatIP(d)  # cosine similarity
        vector_store = FaissVectorStore(faiss_index=faiss_index)
        storage_context = StorageContext.from_defaults(vector_store=vector_store)
        index = GPTVectorStoreIndex.from_documents(documents, storage_context=storage_context, service_context=service_context)

DEBUG:llama_index.readers.file.base:> [SimpleDirectoryReader] Total files added: 1
[make index] - start.
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: 結束バンド 後藤ひとりは友達を作れない陰キャでいつも一人で過ごしていたが、中学時代にテレビのイ...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk:  文化祭ライブ 夏休みに入り、後藤ひとりは知り合いも増えていたが、自分から遊びに誘うことができ...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk:  デモ審査 未確認ライオットに参加するためには、デモ審査、ウェブ投票、ライブ審査を通過して、最...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: テレビアニメ 2022年テレビアニメ化。2022年10月8日よりTOKYO MX、BS11ほか...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: ひとり)...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: ひとり)...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: 秀華高校に通う女子。桃色の髪を無造作に伸ばし、いつもジャージを身につけている。自他共に認める引...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: 放題で前髪で目をつねに隠している。それに加えて野暮ったいジャージ姿でいるため気づかれていないが...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk:   伊地知 虹夏 (いじち にじか)
下北沢高校に通う女子。後藤ひとりより1学年上。ライトイエ...
DEBUG:lla

# クエリエンジンの作成

In [4]:
with timer("make query engine"):
    query_engine = index.as_query_engine(verbose=True)

[make query engine] - start.
[make query engine] - done in 0.0s.


# 質問する

In [5]:
prompt = textwrap.dedent(
    """
    ぼっちちゃんの苦手なことを教えてください
    """
)

with timer("ask question"):
    res = query_engine.query(prompt)

[ask question] - start.
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/embeddings
DEBUG:openai:api_version=None data='{"input": [" \\u307c\\u3063\\u3061\\u3061\\u3083\\u3093\\u306e\\u82e6\\u624b\\u306a\\u3053\\u3068\\u3092\\u6559\\u3048\\u3066\\u304f\\u3060\\u3055\\u3044 "], "model": "text-embedding-ada-002", "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/embeddings HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=35 request_id=1cf105779749deca0ffcc45f52e4d13f response_code=200
DEBUG:llama_index.indices.utils:> Top 2 nodes:
> [Node b0280fee-a982-472b-b26a-833dde8522da] [Similarity score:             0.818654] 秀華高校に通う女子。桃色の髪を無造作に伸ばし、いつもジャージを身につけている。自他共に認める引きこもり一歩手前の「陰キャ」で、承認欲求が人一倍強いにもかかわらず、臆病な性格で人と接するのを極度に...
> [Node 682014cb-e097-417d-a445-4dafbc12b942] [Similarity score:             0.811404] ひとり)...
INFO:

In [6]:
print(res)

ぼっちちゃんは人と接することが極度に苦手である。


# TODO
- llama hubでPDF, Twitter、Slack、Wiki を引っ張ってこれるか試す
- notionはできる？
- 画像の扱いを整理する

# indexを保存

In [ ]:
index.storage_context.persist()

# indexをロード

In [ ]:
from llama_index import StorageContext, load_index_from_storage

storage_context = StorageContext.from_defaults(persist_dir="./storage")
index = load_index_from_storage(storage_context)

In [ ]:
CLOUD_URL = "https://clinicalqa-backend-74qlwzbjrq-uc.a.run.app"
LOCAL_URL = "http://backend:9000"

In [ ]:
def run_health_check(cloud=False):
    url = f"{CLOUD_URL}/health" if cloud else f"{LOCAL_URL}/health"
    res = requests.get(url,
                         #headers={'authorization': f'Bearer {token}'}
                        )
    #res_json = res.json()
    return res

In [ ]:
res = run_health_check(cloud=True)
#res_json = res.json()
res

In [ ]:
res.json()

In [ ]:
def run_predict(payload, cloud=False):
    url = f"{CLOUD_URL}/predict" if cloud else f"{LOCAL_URL}/predict"
    res = requests.post(url, json=payload,
                         #headers={'authorization': f'Bearer {token}'}
                        )
    #res_json = res.json()
    return res

In [ ]:
%%time

pred_payload = {
    "gcs_data_dir": "data/20230212_2135",
}
res = run_predict(payload=pred_payload, cloud=True)
res

In [ ]:
res.json()

In [ ]:
res_json = res.json()
res_json

In [ ]:
res_json["data"]["status"]

In [ ]:
job_status_payload = {
    "gcs_data_dir": "data/20230212_2135",
}

In [ ]:
res = requests.post(f"http://backend:9000/job_status", json=job_status_payload,
                     #headers={'authorization': f'Bearer {token}'}
                    )
res_json = res.json()
res_json